# Feature Generator training: flow-based learning for Darshan trace

This is for a demonstration of how to train Feture Generator for Darshan trace.

In [ ]:
from __future__ import print_function

import argparse

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.utils.data
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.nn import functional as F

# from torchvision import datasets, transforms
# from torchvision.utils import save_image

In [ ]:
class ResNet_block(torch.nn.Module):
    def __init__(self, n, act=torch.nn.LeakyReLU()):
        super().__init__()
        self.module = torch.nn.Sequential(
            torch.nn.Linear(n, n), torch.nn.LeakyReLU(), torch.nn.Linear(n, n),
        )
        self.act = act

    def forward(self, inputs):
        x = self.module(inputs)
        return self.act(x + inputs)

In [ ]:
# https://github.com/pytorch/examples/blob/master/vae/main.py
class VAE(nn.Module):
    def __init__(self, indim, outdim, nh=8, nz=4):
        super(VAE, self).__init__()
        self.indim = indim
        self.outdim = outdim

        self.fc1 = nn.Linear(self.indim, nh)
        self.fc21 = nn.Linear(nh, nz)
        self.fc22 = nn.Linear(nh, nz)
        self.fc3 = nn.Linear(nz, nh)
        self.fc4 = nn.Linear(nh, self.outdim)

    def encode(self, x):
        h1 = F.leaky_relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h3 = F.leaky_relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, self.indim))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [ ]:
# Reconstruction + KL divergence losses summed over all elements and batch
def vae_loss_function(recon_x, x, mu, logvar, dim=20, alpha=1.0):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, dim), reduction="sum")
    MSE = F.mse_loss(recon_x, x.view(-1, dim), reduction="sum")
    # RMSE = torch.sum(RMSELoss(recon_x, x.view(-1, 20)))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return MSE + alpha * KLD

In [ ]:
# https://discuss.pytorch.org/t/rmse-loss-function/16540/2
def RMSELoss(yhat, y):
    return torch.sqrt(torch.mean((yhat - y) ** 2) + torch.finfo(torch.float32).eps)

In [ ]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar, alpha=1.0):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 7), reduction="sum")
    MSE = F.mse_loss(recon_x, x.view(-1, 7), reduction="sum")
    # RMSE = torch.sum(RMSELoss(recon_x, x.view(-1, 20)))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return MSE + alpha * KLD

In [ ]:
class Generator(nn.Module):
    def __init__(self, indim, outdim):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(indim, 64),
            ResNet_block(64),
            ResNet_block(64),
            torch.nn.Linear(64, 32),
            ResNet_block(32),
            ResNet_block(32),
            torch.nn.Linear(32, outdim),
            nn.Tanh(),
        )

    def forward(self, z):
        return self.model(z)


class Discriminator(nn.Module):
    def __init__(self, indim, outdim=1):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(indim, 64),
            ResNet_block(64),
            ResNet_block(64),
            torch.nn.Linear(64, 32),
            ResNet_block(32),
            ResNet_block(32),
            torch.nn.Linear(32, outdim),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
parser = argparse.ArgumentParser(description="FG training")
parser.add_argument(
    "--batch-size",
    type=int,
    default=128,
    metavar="N",
    help="input batch size for training",
)
parser.add_argument(
    "--epochs",
    type=int,
    default=1000,
    metavar="N",
    help="number of epochs to train (default: 10)",
)
parser.add_argument(
    "--no-cuda", action="store_true", default=False, help="disables CUDA training"
)
parser.add_argument(
    "--seed", type=int, default=1, metavar="S", help="random seed (default: 1)"
)
parser.add_argument(
    "--log-interval",
    type=int,
    default=1000,
    metavar="N",
    help="how many batches to wait before logging training status",
)

args = parser.parse_args(["--batch-size=32"])
args.cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if args.cuda else "cpu")

## App list
* Huge:
    'PPP_Hierarchical.mpi.3d', 'cholla.paris-cuda', 'hacc_p3m',
   'lbpm_random_force_simulator', 'main_parallel', 'nekrs',
   'ramses3d', 'sigma.cplx.x', 'tusas', 'xgc-es-cpp-gpu'
* Large:
    'hf_summit_nvblas.x', 'hf_summit_oblate.x', 'lalibe',
    'main_parallel', 'pmemd.cuda.MPI', 'prog_ccm_ex_summit_nat.exe',
    'python', 's3d.x', 'xgc-es-cpp-gpu', 'xspecfem3D'
* Medium:
    'dirac.x', 'epw.x', 'lalibe', 'ngp', 'rmg-gpu', 's3d.x','xspecfem3D'


In [ ]:
## parameters
dtype = "huge"  #'huge', large', 'medium'
APP = 2
NSAMPLES = 100
BATCH = 32
n_flows = 10
modelname = "vae"

In [ ]:
if dtype == "huge":
    nlen, nclass, DIM = 26, 10, 75 + 7
if dtype == "large":
    nlen, nclass, DIM = 220, 10, 60 + 7
if dtype == "medium":
    # nlen, nclass = 945, 12
    nlen, nclass, DIM = 417, 7, 25 + 7

In [ ]:
print(">>> Parameters")
for pname in ["dtype", "DIM", "APP", "NSAMPLES", "BATCH", "n_flows", "modelname"]:
    print("%s: %r" % (pname, eval(pname)))
assert APP < nclass

In [ ]:
x = np.load("../data/train_x_%s_%d.npy" % (dtype, nlen))
y = np.load("../data/train_y_%s_%d.npy" % (dtype, nlen))

lb = np.load("../data/train_lb_%s_%d.npy" % (dtype, nlen))
app = np.load("../data/train_classes_%s_%d_%d.npy" % (dtype, nlen, nclass))
print(x.shape, y.shape, lb.shape, app.shape)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.0001)
print(len(X_train), len(y_train), len(X_test), len(y_test))
assert DIM == x.shape[1]

In [ ]:
xapp = np.zeros((len(x), len(app)), dtype=np.float32)
xapp[np.arange(len(x)), lb] = 1.0
xapp.shape, x.shape, y.shape

In [ ]:
plt.figure(figsize=[10, 4])
plt.subplot(1, 2, 1)
plt.pcolor(xapp)
plt.colorbar()
plt.subplot(1, 2, 2)
plt.pcolor(x)
plt.colorbar()

In [ ]:
def f_density(x, p):
    device = x.device
    n, d = p.shape
    assert len(x[0, :]) == d

    y = torch.zeros_like(x).to(device)
    for i in range(len(x)):
        y[i, :] = (1 - torch.erf((p - x[i, :]) * 10).abs()).sum(axis=0) / n
    return y


def log_f_density(x, p):
    n, d = p.shape
    y = f_density(x, p)
    return torch.log(y.sum(axis=1) / d + np.finfo(np.float).eps)

In [ ]:
p_list = list()
for k in range(nclass):
    s = 0
    x_list = list()
    y_list = list()
    plt.figure(figsize=[12, 3])
    ax1 = plt.subplot(1, 2, 1)
    ax2 = plt.subplot(1, 2, 2)
    for i, enc in enumerate(xapp):
        if enc[k] == 1.0:
            ax1.plot(x[i, :], ".-", label="Original")
            ax2.plot(y[i, :], ".-", label="Original")
            x_list.append(x[i, :])
            y_list.append(y[i, :])
            s += 1
    px = np.array(x_list)
    py = np.array(y_list)
    p_list.append((px, py))
    # print (px.shape)
    ax1.set_ylim([0, 1.1])
    ax2.set_ylim([0, 1.1])
    plt.suptitle("%s (%d)" % (app[k], s))

[(x[0].shape, x[1].shape) for x in p_list]

In [ ]:
# lx = np.linspace(0, 1, int(1e2)+1)
# lxx = np.vstack((lx,lx,lx)).T
# print (lxx.shape)

# z = (f_density(torch.Tensor(lxx), torch.Tensor(x_list)[:,4:7])).numpy()
# print (z.shape)

# plt.figure()
# for i in range(len(z[0,:])):
#     plt.plot(lx, z[:,i], label=i)
# plt.legend()

In [ ]:
# ms = torch.Tensor(x_list)
# n1 = D.Normal(ms[0,:], torch.ones(7)*0.01)
# n2 = D.Normal(ms[1,:], torch.ones(7)*0.01)

# plt.plot(ms[1,:], 's-', c='r');
# samples = n2.sample([10]).detach().numpy()
# plt.plot(samples.T, '-', c='b', alpha=0.2);

## Training

In [ ]:
import torch.distributions as D
from normalizing_flows import NormalizingFlow
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau
from utils import random_normal_samples

In [ ]:
px, py = p_list[APP]
losses = []

if modelname == "flow":
    target_density = lambda z: log_f_density(z, torch.Tensor(px).to(z.device)[:, -DIM:])

    model = NormalizingFlow(DIM, n_flows).to(device)

    # RMSprop is what they used in renzende et al
    optimizer = torch.optim.RMSprop(params=model.parameters(), lr=0.001, momentum=0.1)

    scheduler = ReduceLROnPlateau(optimizer, "min", patience=200)
elif modelname == "gan":
    loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss
    model = Generator(1, DIM).to(device)

    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2_000, gamma=0.1)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, "min", patience=200
    )

    discriminator = Discriminator(DIM, 1).to(device)
    optimizer_D = optim.Adam(discriminator.parameters(), lr=1e-3)
    scheduler_D = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer_D, "min", patience=200
    )
    adversarial_loss = torch.nn.BCELoss()
elif modelname == "vae":
    model = VAE(1, DIM, nh=32, nz=16)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2_000, gamma=0.1)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, "min", patience=2000
    )

In [ ]:
training_data = torch.utils.data.TensorDataset(torch.tensor(px))

kwargs = {"num_workers": 1, "pin_memory": True} if args.cuda else {}
sampler = torch.utils.data.RandomSampler(
    training_data, replacement=True, num_samples=BATCH
)
train_loader = torch.utils.data.DataLoader(
    training_data, batch_size=BATCH, drop_last=False, sampler=sampler, **kwargs
)

print(len(train_loader))

In [ ]:
print("Model params")
print("-" * 20)
num_params = 0
for k, v in model.state_dict().items():
    print("%20s\t%20s\t%10d" % (k, list(v.shape), v.numel()))
    num_params += v.numel()
print("-" * 50)
print("%20s\t%20s\t%10d" % ("Total", "", num_params))
print("All (total, MB): %d %g" % (num_params, num_params * 4 / 1024 / 1024))

In [ ]:
for iter_ in range(5000):
    model.train()
    if iter_ % 100 == 0:
        print("Iteration {}".format(iter_))

    if modelname == "flow":
        samples = Variable(random_normal_samples(BATCH, dim=DIM)).to(device)

        z_k, sum_log_det = model(samples * 0.01)
        log_p_x = target_density(z_k)

        # Reverse KL since we can evaluate target density but can't sample
        loss = (-sum_log_det - (log_p_x)).mean()

    elif modelname == "gan":
        samples = Variable(random_normal_samples(BATCH, dim=1)).to(device)

        (lab,) = next(iter(train_loader))
        lab = lab.to(device)

        samples = Variable(random_normal_samples(BATCH, dim=1)).to(device)
        recon_batch = model(samples)

        valid = Variable(torch.Tensor(BATCH, 1).fill_(1.0), requires_grad=False)
        fake = Variable(torch.Tensor(BATCH, 1).fill_(0.0), requires_grad=False)
        loss = adversarial_loss(discriminator(recon_batch), valid)
    elif modelname == "vae":
        samples = Variable(random_normal_samples(BATCH, dim=1)).to(device)

        (lab,) = next(iter(train_loader))
        lab = lab.to(device)

        recon_batch, mu, logvar = model(samples)
        loss = vae_loss_function(recon_batch, lab, mu, logvar, dim=DIM)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step(loss)

    losses.append(loss.item())

    if modelname == "gan":
        ## Train Discriminator
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(discriminator(lab), valid)
        fake_loss = adversarial_loss(discriminator(recon_batch.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

    if iter_ % 10 == 0:
        print("Loss {} LR: {}".format(loss.item(), optimizer.param_groups[0]["lr"]))

In [ ]:
# Look at the learning
plt.plot(losses)
plt.yscale("log")

In [ ]:
print("Last loss:", losses[-1])

### We generate a training set

In [ ]:
model.eval()
if modelname == "flow":
    samples = Variable(random_normal_samples(NSAMPLES, dim=DIM)).to(device)
    z_k = (model.sample(samples)).detach().cpu().numpy()
elif modelname == "gan":
    samples = Variable(random_normal_samples(NSAMPLES, dim=1)).to(device)
    z_k = (model(samples)).detach().cpu().numpy()
elif modelname == "vae":
    samples = Variable(random_normal_samples(NSAMPLES, dim=1)).to(device)
    recon_batch = model(samples)
    z_k = recon_batch[0].detach().cpu().numpy()

print(z_k.shape)
plt.scatter(z_k[:, 0], z_k[:, 1], s=2)

In [ ]:
import seaborn as sns

for i in range(5):
    plt.figure()
    sns.jointplot(z_k[:, i], z_k[:, i + 1], kind="kde")

In [ ]:
for i in range(DIM):
    plt.hist(z_k[:, i], bins=50, alpha=0.7, label=i)
# plt.legend()

In [ ]:
plt.figure()
for s in z_k:
    plt.plot(s, alpha=0.7)

In [ ]:
xx = x[lb == APP, :]
plt.plot(xx.T);

In [ ]:
plt.figure()
z = np.mean(z_k, axis=0)
z_fit = (z - z.min()) / (z.max() - z.min())
plt.plot(z_fit, label="gen")
plt.plot(np.mean(xx, axis=0), label="org")
plt.legend()

In [ ]:
ids = np.random.choice(range(len(py)), NSAMPLES)
z_k_y = py[ids, :]

print(z_k.shape, z_k_y.shape)

In [ ]:
fname = "flowmodel_%s_%d_DIM%d_app%d_%s.torch" % (dtype, nlen, DIM, APP, modelname)
torch.save(model.state_dict(), fname)
print("Model saved:", fname)

np.save("_train_x_%s_%d_DIM%d_app%d_%s.npy" % (dtype, nlen, DIM, APP, modelname), z_k)
np.save("_train_y_%s_%d_DIM%d_app%d_%s.npy" % (dtype, nlen, DIM, APP, modelname), z_k_y)

## Merging

In [ ]:
lx = list()
ly = list()
lb_list = list()

all_ready = True
for APP in range(nclass):
    try:
        _x = np.load(
            "_train_x_%s_%d_DIM%d_app%d_%s.npy" % (dtype, nlen, DIM, APP, modelname)
        )
        _y = np.load(
            "_train_y_%s_%d_DIM%d_app%d_%s.npy" % (dtype, nlen, DIM, APP, modelname)
        )

        lx.append(_x)
        ly.append(_y)
        lb_list.append([APP,] * len(_x))
    except:
        all_ready &= False
        print(">>> Not ready:", dtype, APP)
        pass

if all_ready:
    xx = np.concatenate(lx)
    yy = np.concatenate(ly)
    ll = np.concatenate(lb_list)
    print("FG:", xx.shape, yy.shape, ll.shape)

    np.save("flow_train_x_%s_%d_DIM%d_%s.npy" % (dtype, nlen, DIM, modelname), xx)
    np.save("flow_train_y_%s_%d_DIM%d_%s.npy" % (dtype, nlen, DIM, modelname), yy)
    np.save("flow_train_lb_%s_%d_DIM%d_%s.npy" % (dtype, nlen, DIM, modelname), ll)